# Proyecto SQL

La pandemia del coronavirus alteró significativamente la vida cotidiana, afectando especialmente las actividades sociales y culturales. Los cafés, centros comerciales y otros puntos de encuentro dejaron de ser la opción para muchas personas, mientras que quedarse en casa y leer libros se convirtió en una de las formas más comunes de entretenimiento. Este cambio en los hábitos de consumo trajo consigo una creciente demanda de aplicaciones dedicadas a los amantes de los libros, lo que llevó a las startups a enfocarse en desarrollar productos innovadores para este nuevo mercado.

El proyecto que presentamos aquí se basa en el análisis de una base de datos que contiene información clave sobre libros, autores, editoriales, calificaciones y reseñas de usuarios. A través de consultas SQL, se busca extraer información valiosa de esta base de datos para ayudar a generar una propuesta de valor para un nuevo producto en el mercado editorial. Este análisis se centrará en obtener respuestas a preguntas clave sobre las preferencias de los lectores, el desempeño de los libros en el mercado, y las tendencias en las calificaciones y reseñas de los usuarios.

Para llevar a cabo este análisis, se utilizarán las siguientes tablas de datos:

* Books: Contiene información sobre los libros, incluyendo el título, autor, número de páginas, fecha de publicación y editorial.
    
* Authors: Detalles sobre los autores, como su identificación y nombre.
    
* Publishers: Información sobre las editoriales, identificadas por su ID y nombre.
    
* Ratings: Contiene las calificaciones que los usuarios asignan a los libros, junto con su nombre de usuario.
    
* Reviews: Incluye las reseñas textuales realizadas por los usuarios, con su nombre de usuario y el contenido de la reseña.

El objetivo de este proyecto es responder a diversas preguntas de análisis mediante el uso de consultas SQL, lo cual permitirá conocer más a fondo las tendencias del mercado editorial en un contexto post-pandemia. A lo largo de este estudio, se realizarán diversas consultas para identificar patrones de publicación, calificación, y reseña que aporten valor al desarrollo de un nuevo producto.

Las tareas a realizar incluyen la identificación de libros publicados después de una fecha determinada, la obtención de calificaciones promedio y reseñas de libros, la identificación de la editorial con mayor número de publicaciones relevantes, y el análisis de las calificaciones más altas obtenidas por autores.

**Objetivos del estudio**

* Analizar el comportamiento de publicación de libros después del 1 de enero de 2000.

* Determinar el número de reseñas de usuarios y la calificación promedio para cada libro.

* Identificar la editorial que ha publicado el mayor número de libros con más de 50 páginas.

* Encontrar el autor con la más alta calificación promedio de libros (considerando solo los libros con al menos 50 calificaciones).

* Calcular el número promedio de reseñas de texto entre los usuarios que han calificado más de 50 libros.

In [1]:
#Importar librerías
import pandas as pd
from sqlalchemy import create_engine

In [2]:
#Conectarse a la base de datos
db_config = {'user': 'practicum_student',         # nombre de usuario
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # contraseña
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # puerto de conexión
             'db': 'data-analyst-final-project-db'}          # nombre de la base de datos

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
# Consultas SQL
query_books = "SELECT * FROM books LIMIT 5;"
query_authors = "SELECT * FROM authors LIMIT 5;"
query_publishers = "SELECT * FROM publishers LIMIT 5;"
query_ratings = "SELECT * FROM ratings LIMIT 5;"
query_reviews = "SELECT * FROM reviews LIMIT 5;"

# Imprimir las primeras filas de cada tabla
books = pd.io.sql.read_sql(query_books, con = engine)
authors = pd.io.sql.read_sql(query_authors, con = engine)
publishers= pd.io.sql.read_sql(query_publishers, con = engine)
ratings = pd.io.sql.read_sql(query_ratings, con = engine)
reviews = pd.io.sql.read_sql(query_reviews, con = engine)

print(books.head())
print(authors.head())
print(publishers.head())
print(ratings.head())
print(reviews.head())


   book_id  author_id                                              title  \
0        1        546                                       'Salem's Lot   
1        2        465                 1 000 Places to See Before You Die   
2        3        407  13 Little Blue Envelopes (Little Blue Envelope...   
3        4         82  1491: New Revelations of the Americas Before C...   
4        5        125                                               1776   

   num_pages publication_date  publisher_id  
0        594       2005-11-01            93  
1        992       2003-05-22           336  
2        322       2010-12-21           135  
3        541       2006-10-10           309  
4        386       2006-07-04           268  
   author_id                          author
0          1                      A.S. Byatt
1          2  Aesop/Laura Harris/Laura Gibbs
2          3                 Agatha Christie
3          4                   Alan Brennert
4          5        Alan Moore/David   Llo

In [4]:
#Número de libros publicados después del 1 de enero de 2000
query_num_books = """
SELECT COUNT(*) AS num_books
FROM books
WHERE publication_date > '2000-01-01';
"""

#Ejecutar consulta SQL utilizando pandas
num_books = pd.io.sql.read_sql(query_num_books, con = engine)

#Mostrar resultados
print(num_books.head())

   num_books
0        819


Se han publicado 819 libros después del 1 de enero de 2000. Esto indica un nivel considerable de actividad en la publicación de libros en el período reciente.

In [5]:
#Número de reseñas de usuarios y calificación promedio para cada libro
query_reviews_ratings = """
SELECT books.book_id, books.title, COUNT(reviews.review_id) AS num_reviews, AVG(ratings.rating) AS avg_rating
FROM books
LEFT JOIN reviews ON books.book_id = reviews.book_id
LEFT JOIN ratings ON books.book_id = ratings.book_id
GROUP BY books.book_id, books.title;
"""

#Ejecutar consulta SQL utilizando pandas
reviews_ratings = pd.io.sql.read_sql(query_reviews_ratings, con = engine)

#Mostrar resultados
print(reviews_ratings.head())


   book_id                                              title  num_reviews  \
0      652          The Body in the Library (Miss Marple  #3)            4   
1      273                                          Galápagos            4   
2       51                           A Tree Grows in Brooklyn           60   
3      951  Undaunted Courage: The Pioneering First Missio...            4   
4      839                                        The Prophet           28   

   avg_rating  
0    4.500000  
1    4.500000  
2    4.250000  
3    4.000000  
4    4.285714  


Cada libro tiene un número variado de reseñas y una calificación promedio diferente. Por ejemplo, el libro "A Tree Grows in Brooklyn" tiene 60 reseñas con una calificación promedio de 4.25. Esto proporciona una visión clara de la participación del usuario y la percepción de cada libro.

In [6]:
#Editorial que ha publicado el mayor número de libros con más de 50 páginas
query_publishers_books = """
SELECT publishers.publisher, COUNT(books.book_id) AS num_books
FROM books
JOIN publishers ON books.publisher_id = publishers.publisher_id
WHERE books.num_pages > 50
GROUP BY publishers.publisher
ORDER BY num_books DESC
LIMIT 1;
"""

#Ejecutar consulta SQL utilizando pandas
publishers_books = pd.io.sql.read_sql(query_publishers_books, con = engine)

#Mostrar resultados
print(publishers_books.head())

       publisher  num_books
0  Penguin Books         42


La editorial "Penguin Books" ha publicado el mayor número de libros con más de 50 páginas, con un total de 42 libros. Esta información es útil para identificar a los principales editores en el mercado de libros de mayor contenido.

In [7]:
#Autor con la más alta calificación promedio del libro (con al menos 50 calificaciones)
query_top_author = """
SELECT authors.author, AVG(ratings.rating) AS avg_rating
FROM books
JOIN authors ON books.author_id = authors.author_id
JOIN ratings ON books.book_id = ratings.book_id
GROUP BY authors.author
HAVING COUNT(ratings.rating_id) >= 50
ORDER BY avg_rating DESC
LIMIT 1;
"""

#Ejecutar consulta SQL utilizando pandas
top_author = pd.io.sql.read_sql(query_top_author, con = engine)

#Mostrar resultados
print(top_author.head())

           author  avg_rating
0  Diana Gabaldon         4.3


El autor con la calificación promedio más alta para sus libros (considerando solo aquellos con al menos 50 calificaciones) es Diana Gabaldon, con una calificación promedio de 4.3. Este hallazgo puede ser útil para destacar a los autores más valorados por los lectores.

In [8]:
#Número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros
query_avg_text_reviews = """
WITH user_reviews AS (
    SELECT username, COUNT(review_id) AS num_reviews
    FROM reviews
    GROUP BY username
),
user_ratings AS (
    SELECT username, COUNT(rating_id) AS num_ratings
    FROM ratings
    GROUP BY username
)
SELECT AVG(user_reviews.num_reviews) AS avg_text_reviews
FROM user_reviews
JOIN user_ratings ON user_reviews.username = user_ratings.username
WHERE user_ratings.num_ratings > 50;
"""

#Ejecutar consulta SQL utilizando pandas
avg_text_reviews = pd.io.sql.read_sql(query_avg_text_reviews, con = engine)

#Mostrar resultados
print(avg_text_reviews.head())

   avg_text_reviews
0         24.333333


En promedio, los usuarios que han calificado más de 50 libros han escrito aproximadamente 24 reseñas de texto. Esto indica un alto nivel de participación de estos usuarios activos en el servicio.